In [108]:
import pickle
import os
import time

In [195]:
SampleDirectory = "../RAW_SEQUENCES/"
recursive=True

In [196]:
os.path.isdir(SampleDirectory)

True

In [281]:
assert os.path.isdir(SampleDirectory), "%r is not a directory." % SampleDirectory

directories = [x[0] for x in os.walk(SampleDirectory)]
FileNames = [x[2] for x in os.walk(SampleDirectory)]

SamplePathList = []
SampleNameList = []

if recursive:
    for i in range(0, len(directories)):
        [SamplePathList.append(os.path.join(directories[i], j)) for j in FileNames[i] if "fastq" in j]
        [SampleNameList.append(j) for j in FileNames[i] if "fastq" in j]
else:
    [SamplePathList.append(os.path.join(directories[0], j)) for j in FileNames[0] if "fastq" in j]
    [SampleNameList.append(j) for j in FileNames[0] if "fastq" in j]

assert len(SamplePathList) is not 0, "No samples found in %r" % SampleDirectory

In [292]:
print([x[0] for x in os.walk(SampleDirectory)])
[x[2] for x in os.walk(SampleDirectory)]

['../RAW_SEQUENCES/', '../RAW_SEQUENCES/batch3_extracted_4-14-17', '../RAW_SEQUENCES/Even_Deeper', '../RAW_SEQUENCES/Even_Deeper/IND03975-43224285', '../RAW_SEQUENCES/IND03977-43208468']


[['.DS_Store',
  'IND03978_S75_L001_R1_001.fastq.gz',
  'IND03978_S75_L001_R2_001.fastq.gz'],
 ['IND01817_S41_L001__KIR_1.fastq.gz',
  'IND01817_S41_L001__KIR_2.fastq.gz',
  'IND01818_S42_L001__KIR_1.fastq.gz',
  'IND01818_S42_L001__KIR_2.fastq.gz',
  'IND01819_S43_L001__KIR_1.fastq.gz',
  'IND01819_S43_L001__KIR_2.fastq.gz',
  'IND01820_S44_L001__KIR_1.fastq.gz',
  'IND01820_S44_L001__KIR_2.fastq.gz',
  'IND01821_S45_L001__KIR_1.fastq.gz',
  'IND01821_S45_L001__KIR_2.fastq.gz',
  'IND01822_S46_L001__KIR_1.fastq.gz',
  'IND01822_S46_L001__KIR_2.fastq.gz',
  'IND01823_S47_L001__KIR_1.fastq.gz',
  'IND01823_S47_L001__KIR_2.fastq.gz',
  'IND01824_S48_L001__KIR_1.fastq.gz',
  'IND01824_S48_L001__KIR_2.fastq.gz',
  'IND01825_S49_L001__KIR_1.fastq.gz',
  'IND01825_S49_L001__KIR_2.fastq.gz',
  'IND01826_S50_L001__KIR_1.fastq.gz',
  'IND01826_S50_L001__KIR_2.fastq.gz',
  'IND01827_S51_L001__KIR_1.fastq.gz',
  'IND01827_S51_L001__KIR_2.fastq.gz',
  'IND01828_S52_L001__KIR_1.fastq.gz',
  'IND018

In [283]:

def match_finder(SampleNameList, SamplePathList):
    FoundMatches = {}

    while len(SampleNameList) > 0:
        sample = SampleNameList[0]
        SubStr = ""
        LastStr = ""
    
        for char in sample:
            LastStr = SubStr
            SubStr += char
            MatchNum = sum([i.count(SubStr) for i in SampleNameList])
        
            if MatchNum == 1:
                MatchIndices = [i for i in range(0, len(SampleNameList)) if LastStr in SampleNameList[i]]
            
                if len(MatchIndices) is not 2:
                    print("%s matches found for %s." % (len(MatchIndices), sample))
                    print("Removing %s from match consideration." % sample)
                    del SampleNameList[0]
                    del SamplePathList[0]
                    break
            
                MatchNames = [SampleNameList[i] for i in MatchIndices]
                MatchPaths = [SamplePathList[i] for i in MatchIndices]
                FoundMatches[LastStr] = {MatchNames[i]: MatchPaths[i] for i in range(0,len(MatchNames))}
                for i in sorted(MatchIndices, reverse=True):
                    del SampleNameList[i]
                    del SamplePathList[i]
                break
    return(FoundMatches)

1 matches found for IND03977_S74_L001_R1_001.fastq.gz.
Removing IND03977_S74_L001_R1_001.fastq.gz from match consideration.
0.00074005126953125


In [285]:
indexList = [1]
SampleNameList = ["a", "b", "c"]
SampleNameList[indexList]

TypeError: list indices must be integers or slices, not list

In [284]:
FoundMatches

{'IND03975_S72_L001_R': {'IND03975_S72_L001_R1_001.fastq.gz': '../RAW_SEQUENCES/Even_Deeper/IND03975-43224285/IND03975_S72_L001_R1_001.fastq.gz',
  'IND03975_S72_L001_R2_001.fastq.gz': '../RAW_SEQUENCES/Even_Deeper/IND03975-43224285/IND03975_S72_L001_R2_001.fastq.gz'},
 'IND03978_S75_L001_R': {'IND03978_S75_L001_R1_001.fastq.gz': '../RAW_SEQUENCES/IND03978_S75_L001_R1_001.fastq.gz',
  'IND03978_S75_L001_R2_001.fastq.gz': '../RAW_SEQUENCES/IND03978_S75_L001_R2_001.fastq.gz'}}

In [4]:
my_list = ['one']
my_list[0:-1]

[]